In [1]:
# import packages
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import socket
import ipywidgets as widgets
from IPython.display import display
import os
from se4g_dir import set_the_working_directory
set_the_working_directory()

Current directory:  c:\Users\Utente\Documents\GitHub\SE4GEO-Lab\db_helper
Current working directory: c:\Users\Utente\Documents\GitHub\SE4GEO-Lab


Connection

In [2]:
from se4g_helper import connect_right_now, connect_with_sqlalchemy

In [ ]:
data_type = ['VARCHAR',
            'VARCHAR',
            'FLOAT',
            'CHAR(2)',
            'VARCHAR',
            'VARCHAR',
            'VARCHAR',
            'VARCHAR',
            'FLOAT',
            'FLOAT',
            'FLOAT']

df_columns=[
        'station_code',
        'station_name',
        'station_altitude',
        'network_countrycode',
        'pollutant',
        'value_datetime_begin',
        'value_datetime_end',
        'value_datetime_updated',
        'value_numeric',
        'samplingpoint_x',
        'samplingpoint_y'
    ]

### Create table

In [5]:
conn = connect_right_now()

# ANGELICA'S WORK IN PROGRESS
cur = conn.cursor()
cur.execute(
    '''CREATE TABLE IF NOT EXISTS station
        (station_code VARCHAR PRIMARY KEY, 
        station_name TEXT,
        station_altitude FLOAT,
        network_countrycode TEXT,
        samplingpoint_x FLOAT,
        samplingpoint_y FLOAT)'''
            )
print('created')
conn.commit()
cur.close()

connected with  localhost
created


In [31]:
conn = connect_right_now()

# ANGELICA'S WORK IN PROGRESS
cur = conn.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS pollutant_detection
 (station_code VARCHAR , 
 pollutant TEXT, 
 value_datetime_begin TEXT, 
 value_datetime_end TEXT,
 value_numeric FLOAT,
 PRIMARY KEY (station_code, pollutant, value_datetime_begin))''')
print('created')
conn.commit()
cur.close()

conn.close()


connected with  localhost
created


### Alternative table creation 
iLory x Angie

In [11]:
# SELECT THE NAME OF THE CUSTOM TABLE
table_name='se4g_dashboard2' 

# INSERT HERE THE TYPE OF EACH COLUMN OF THE CUSTOM TABLE
data_type = data_type

# DEFINE HERE THE NAME OF EACH COLUMN OF THE CUSTOM TABLE
df_columns= df_columns

# CREATE THE CUSTOM TABLE
def create_custom_TABLE(table_name, conn,
                        data_type = data_type,
                        df_columns = df_columns,
                                        ):
    def table_exists(table_name, conn):
        cur = conn.cursor()
        cur.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE LOWER(table_name) = LOWER('{table_name}'))")
        exists = cur.fetchone()[0]
        cur.close()
        return exists
    if not table_exists(table_name, conn):

        cur = conn.cursor()
        # In this way we add 'station_code' as PRIMARY KEY 
        column_definitions = [f"{column} {data_type[i]}" if column != 'station_code' else f"{column} {data_type[i]} PRIMARY KEY" for i, column in enumerate(df_columns)]
        #column_definitions = [f"{column} {data_type[i]}" for i, column in enumerate(df_columns)] # old code that does note assign the PRIMARY KEY
        
        create_table_statement = f"CREATE TABLE {table_name} ({', '.join(column_definitions)})"
        print(create_table_statement)

        cur.execute(create_table_statement)
        conn.commit()
        cur.close()
        print('created')
    else:
        exists = table_exists(table_name,conn)
        print(f"The table '{table_name}' exists: {exists}")

create_custom_TABLE('custom_table1', conn)

conn.close()

CREATE TABLE custom_table1 (station_code VARCHAR PRIMARY KEY, station_name VARCHAR, station_altitude FLOAT, network_countrycode CHAR(2), pollutant VARCHAR, value_datetime_begin VARCHAR, value_datetime_end VARCHAR, value_datetime_updated VARCHAR, value_numeric FLOAT)
created


In [ ]:
# YOU CAN USE THIS FUNCTION TO COPY A TABLE INSIDE ANOTHER NEW TABLE e.g. FOR BACKUP PURPOSE (BEFORE DOING A MESS FOR EXAMPLE)
def copy_table_data(connection, source_table_name, destination_table_name):
    cursor = connection.cursor()
    
    # Truncate the destination table to remove existing data (optional)
    cursor.execute(f"TRUNCATE TABLE {destination_table_name};")
    
    # Generate the SQL statement to copy data from the source table to the destination table
    copy_data_query = f"INSERT INTO {destination_table_name} SELECT * FROM {source_table_name};"
    
    # Execute the COPY command
    cursor.execute(copy_data_query)
    
    # Commit the changes
    connection.commit()
    
    # Close the cursor
    cursor.close()

# Specify the names of the source table and destination table
source_table_name = 'se4g_pollution'
destination_table_name = 'se4g_pollution_backup_070623'

# EXECUTE THE FUNCTION
# Call the function to copy the data from the source table to the destination table
copy_table_data(conn, source_table_name, destination_table_name)

# Close the connection
conn.close()


### Fill table

In [ ]:
conn = connect_right_now()

def print_table_data(connection, 
                    source_table_name, 
                    columns = [ 'station_code' , 
                                'station_name', 
                                'station_altitude', 
                                'network_countrycode'
                                ]
                    ):
    cursor = connection.cursor()
    
    # Truncate the destination table to remove existing data (optional)
    cursor.execute(f"TRUNCATE TABLE {destination_table_name};")
    
    # Generate the SQL statement to copy data from the source table to the destination table
    copy_data_query = f"SELECT DISTINCT * FROM {source_table_name};"
    
    # Execute the COPY command
    cursor.execute(copy_data_query)

    # Fetch all the rows
    rows = cursor.fetchall()
    
    # Create a pandas DataFrame from the fetched rows
    df = pd.DataFrame(rows, columns=columns)

    # Group the DataFrame by 'station_code' and filter the groups with more than one unique combination of values
    #filtered_df = df.groupby('station_code').filter(lambda x: len(x.drop_duplicates(subset=['station_name', 'station_altitude', 'network_countrycode'])) > 1)

    # Print the filtered DataFrame
    print(df)

    # Close the cursor
    cursor.close()

# Specify the names of the source table and destination table
table_name = 'station'

print_table_data(conn, table_name)

# Close the connection
conn.close()

In [6]:
conn = connect_right_now()

source_table_name = 'se4g_pollution_main'
destination_table_name = 'station'

def fill_table_data(connection,
                    source_table_name,
                    destination_table_name,
                    columns=['station_code',
                             'station_name',
                             'station_altitude',
                             'network_countrycode',
                             'samplingpoint_x',
                             'samplingpoint_y']
                    ):
    cursor = connection.cursor()

    # Truncate the destination table to remove existing data (optional)
    cursor.execute(f"TRUNCATE TABLE {destination_table_name};")

    # Create a unique index on 'station_code' column
    cursor.execute(f"CREATE UNIQUE INDEX idx_unique_station_code ON {destination_table_name} (station_code);")

    # Generate the SQL statement to copy data from the source table to the destination table
    copy_data_query = f"INSERT INTO {destination_table_name} ({', '.join(columns)}) " \
                      f"SELECT DISTINCT ON (station_code) {', '.join(columns)} FROM {source_table_name};"

    # Execute the COPY command
    cursor.execute(copy_data_query)

    # Commit the changes
    connection.commit()
    print(f'Table {destination_table_name} filled')

    # Close the cursor
    cursor.close()

fill_table_data(conn, source_table_name, destination_table_name)


# Close the connection
conn.close()

connected with  localhost
Table station filled


In [3]:


destination_table_name = 'pollutant_detection'

# ANGELICA'S WORK IN PROGRESS
cur = conn.cursor()
cur.execute(f'''CREATE TABLE IF NOT EXISTS {destination_table_name}
 (station_code VARCHAR , 
 pollutant TEXT, 
 value_datetime_begin TEXT, 
 value_datetime_end TEXT,
 value_numeric FLOAT,
 samplingpoint_x FLOAT,
 samplingpoint_y FLOAT,
 PRIMARY KEY (station_code, pollutant, value_datetime_begin))''')
print('created')
conn.commit()
cur.close()


connected with  localhost  through psycopg2
created


In [6]:
conn = connect_right_now() # time required - 2m44s



source_table_name = 'se4g_pollution_main'


def fill_table_data(connection,
                    source_table_name,
                    destination_table_name,
                    columns=['station_code',
                             'pollutant',
                             'value_datetime_begin',
                             'value_datetime_end',
                             'value_numeric',
                             'samplingpoint_x',
                             'samplingpoint_y']
                    ):
    cursor = connection.cursor()

    # Truncate the destination table to remove existing data (optional)
    cursor.execute(f"TRUNCATE TABLE {destination_table_name};")

    # Create a unique index on 'station_code' column
    cursor.execute(f"CREATE UNIQUE INDEX idx_unique_p_s_v ON {destination_table_name} (station_code, pollutant, value_datetime_begin, value_datetime_end);")

    # Generate the SQL statement to copy data from the source table to the destination table
    copy_data_query = f"INSERT INTO {destination_table_name} ({', '.join(columns)}) " \
                      f"SELECT DISTINCT ON (station_code, pollutant, value_datetime_begin, value_datetime_end) {', '.join(columns)} FROM {source_table_name};"

    # Execute the COPY command
    cursor.execute(copy_data_query)

    # Commit the changes
    connection.commit()
    print(f'Table {destination_table_name} filled')

    # Close the cursor
    cursor.close()

fill_table_data(conn, source_table_name, destination_table_name)

# Close the connection
conn.close()

connected with  localhost  through psycopg2
Table pollutant_detection filled


### Update tables

In [7]:
from se4g_helper import download_DB

conn = connect_right_now()

all_rows = download_DB(connection=conn) 
# time required - 12m51s

connected with  localhost
-----------------------------------------------------------------------
countries ['AD', 'AT', 'BA', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'ES', 'DK', 'EE', 'FI', 'SE']
Downloading: http://discomap.eea.europa.eu/map/fme/latest/AD_SO2.csv
lines[0] network_countrycode,network_localid,network_name,network_namespace,network_timezone,pollutant,samplingpoint_localid,samplingpoint_namespace,samplingpoint_x,samplingpoint_y,coordsys,station_code,station_localid,station_name,station_namespace,value_datetime_begin,value_datetime_end,value_datetime_inserted,value_datetime_updated,value_numeric,value_validity,value_verification,station_altitude,value_unit
Download finished and new_rows assembled
new_rows[:2] unfiltered [('AD0942A', 'Escaldes-Engordany', '1080', 'AD', 'SO2', '2023-06-05 23:00:00+01:00', '2023-06-06 00:00:00+01:00', '+01:00', '0.4'), ('AD0942A', 'Escaldes-Engordany', '1080', 'AD', 'SO2', '2023-06-06 00:00:00+01:00', '2023-06-06 01:00:00+01:00', '+01:00', '0.4'

In [18]:
columns=['station_code', 'pollutant', 'value_datetime_begin', 'value_datetime_end', 'value_numeric']
table_name='pollutant_detection'
print(f"INSERT INTO {table_name} ({', '.join(columns)})")

INSERT INTO pollutant_detection (station_code, pollutant, value_datetime_begin, value_datetime_end, value_numeric)


In [28]:
from datetime import datetime 

def insert_data(table_name, rows, conn, columns):
    cur = conn.cursor()

    # Generate the SQL INSERT statement with specified columns
    insert_statement = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

    '''rows = [
        tuple(
            val.strftime('%Y-%m-%d %H:%M:%S%z') if isinstance(val, datetime) else val
            if val != '' else None  # Replace empty string with None for double precision columns
            for val in row
        )
        for row in rows
    ]'''

    # Execute the INSERT statement for each row
    cur.executemany(insert_statement, rows)

    # Commit the changes and close the cursor
    conn.commit()
    cur.close()


# Update DATABASE
def update_table(
    connection,
    new_table = 'se4g_pollution_db',
    table_name='pollutant_detection',
    columns=['station_code', 'pollutant', 'value_datetime_begin', 'value_datetime_end', 'value_numeric'],
            # station_code,   pollutant,   value_datetime_begin
    ):
    
    
    cur = connection.cursor()
    new_cur = connection.cursor()
    # Generate the SQL SELECT statement
    select_statement = f"SELECT * FROM {table_name}"
    new_select_statement = f"SELECT {', '.join(columns)} FROM {new_table}"
        
    # Execute the SELECT statement
    cur.execute(select_statement)
    new_cur.execute(new_select_statement)

    # Fetch all the results
    results = cur.fetchall()
    new_results = new_cur.fetchall()

    # Convert the results to a set of tuples for comparison
    existing_data = {tuple(row) for row in results}
    new_rows = {tuple(row) for row in new_results}

    # Filter new_rows to only include rows not already present in the table
    filtered_rows = [row for row in new_rows if tuple(row) not in existing_data]

    if len(filtered_rows) == 0:
        print("Nothing to update inside database", table_name)
    elif len(filtered_rows) > 0:

        # Execute the INSERT statement to add the filtered rows
        insert_data(table_name, filtered_rows, connection, columns)
        print("Database", table_name, "updated successfully")

    # Close the cursor
    cur.close()
    new_cur.close()

    return filtered_rows

connected with  localhost


In [27]:
conn = connect_right_now()
from datetime import datetime 

def insert_data(table_name, rows, conn, columns):
    cur = conn.cursor()

    # Generate the SQL INSERT statement with specified columns
    insert_statement = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

    # Execute the INSERT statement for each row
    cur.executemany(insert_statement, rows)

    # Commit the changes and close the cursor
    conn.commit()
    cur.close()


# Update DATABASE
def update_DB(
    new_rows,
    connection,
    table_name='pollutant_detection',
    columns=['station_code', 'pollutant', 'value_datetime_begin', 'value_datetime_end', 'value_numeric'],
    df_columns=df_columns
):
    selected_rows = [tuple(row[df_columns.index(col)] for col in columns) for row in new_rows]

    cur = connection.cursor()

    # Generate the SQL SELECT statement
    select_statement = f"SELECT * FROM {table_name}"

    # Execute the SELECT statement
    cur.execute(select_statement)

    # Fetch all the results
    results = cur.fetchall()

    # Convert the results to a set of tuples for comparison
    existing_data = {tuple(row) for row in results}

    # Filter new_rows to only include rows not already present in the table
    filtered_rows = [row for row in selected_rows if tuple(row) not in existing_data]

    if len(filtered_rows) == 0:
        print("Nothing to update inside database", table_name)
    elif len(filtered_rows) > 0:
        # Execute the INSERT statement to add the filtered rows
        insert_data(table_name, filtered_rows, connection, columns)
        print("Database", table_name, "updated successfully")

    # Close the cursor
    cur.close()

    return filtered_rows

for new_rows in all_rows:
    update_DB(new_rows, conn)

conn.close()


connected with  localhost


UniqueViolation: duplicate key value violates unique constraint "pollutant_detection_pkey"
DETAIL:  Key (pollutant, station_code, value_datetime_begin)=(SO2, AD0942A, 2023-06-05 23:00:00+01:00) already exists.


In [30]:
conn = connect_right_now()

update_table(conn)

conn.close()

connected with  localhost


UniqueViolation: duplicate key value violates unique constraint "pollutant_detection_pkey"
DETAIL:  Key (pollutant, station_code, value_datetime_begin)=(NO, ES1253A, 2023-06-05 23:00:00+01:00) already exists.


# Dashboard

In [3]:
conn = connect_right_now()

# ANGELICA'S WORK IN PROGRESS
cur = conn.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS se4g_new_dashboard
 (network_countrycode VARCHAR , 
 country TEXT,
 pollutant TEXT, 
 value_datetime_begin TEXT, 
 value_numeric FLOAT,
 value_numeric_mean FLOAT,
 PRIMARY KEY (network_countrycode, pollutant, value_datetime_begin))''')
print('created')
conn.commit()
cur.close()

conn.close()

connected with  localhost  through psycopg2
created


In [ ]:
conn = connect_right_now()

def select_rows_not_in_dashboard(connection, source_table='se4g_pollution_main', table_name='se4g_new_dashboard'):
    cur = connection.cursor()

    # Generate the SQL SELECT statement
    select_statement = f'''
        SELECT s.network_countrycode, s.pollutant, s.value_datetime_end, s.value_numeric
        FROM {source_table} s
        LEFT JOIN {table_name} t
        ON s.network_countrycode = t.network_countrycode
        AND s.pollutant = t.pollutant
        AND s.value_datetime_end = t.value_datetime_end
        WHERE t.network_countrycode IS NULL
        AND t.pollutant IS NULL
        AND t.value_datetime_end IS NULL
    '''

    # Execute the SELECT statement
    cur.execute(select_statement)

    # Fetch all the selected rows
    selected_rows = cur.fetchall()

    # Close the cursor
    cur.close()

    return selected_rows

# Call the function to select rows not present in table_dashboard
selected_rows = select_rows_not_in_dashboard(conn)

# Print the selected rows
for row in selected_rows:
    print()

# Close the database connection
conn.close()


In [ ]:
source_df = "se4g_pollution_main.csv" # execution time: 1m23s -> 1m51s
folder_out = 'data_prova'
full_path = os.path.join(folder_out, source_df)

columns = ['pollutant', 'country', 'month_day', 'value_numeric_mean']

country = {'AD': 'Andorra', 'SE': 'Sweden', 'DE': 'Germany', 'CY': 'Undefined', 'BE': 'Belgium',
           'FI': 'Finland', 'ES': 'Spain', 'CZ': 'Czech Republic', 'BG': 'Bulgaria', 'BA': 'Bosnia and Herzegovina',
           'EE': 'Estonia', 'CH': 'Switzerland', 'AT': 'Austria', 'DK': 'Denmark'}
   
if os.path.isfile(full_path):
    dashboard_df = pd.read_csv(full_path)
    # Convert 'value_datetime_end' to datetime objects and extract the day
    datetime_objects = dashboard_df['value_datetime_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z'))
    dashboard_df['month_day'] = datetime_objects.apply(lambda x: x.strftime('%m%d'))

    # Compute daily mean of 'value_numeric' for each 'pollutant' and 'network_countrycode'
    daily_mean = dashboard_df.groupby(['pollutant', 'network_countrycode', 'month_day'])['value_numeric'].mean().reset_index()
    
    # Merge the daily mean back to the original dataframe
    dashboard_df = dashboard_df.merge(daily_mean, on=['pollutant', 'network_countrycode', 'month_day'], suffixes=('', '_mean'))    

    dashboard_df['country'] = dashboard_df['network_countrycode'].map(country)
    dashboard_df = dashboard_df[columns].copy()
    dashboard_df = dashboard_df[dashboard_df['pollutant'] != 'O3']
    dashboard_df = dashboard_df.drop_duplicates().reset_index(drop=True)
    dashboard_df = dashboard_df.sort_values('month_day')

    dashboard_df.to_csv('data_prova/se4g_dashboard_dataset.csv', index=False)
    print("Dataset ",full_path," created")
    
else: 
        print("Dataset ",full_path," does not exist")
        print(os.getcwd())